### Load Colab Notebook settings

**Link Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Try GPU power**

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=9c322a63a2a94c87301b32004f489a0b12ad2a8459ae9670bab89742db4e697e
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.6 GB  | Proc size: 456.8 MB
GPU RAM Free: 11372MB | Used: 69MB | Util   1% | Total 11441MB


In [0]:
#!kill -9 -1

## Train the model

In [23]:
# mettre dans un .py qaund on a trouvé les bons params

import numpy as np
import sys
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as keras
import re

def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

def data_load(directory_name): 
    filenames = sorted_aphanumeric(os.listdir(directory_name))
    imgs = []
    for i, fileNb in enumerate(filenames):
        if(fileNb[0]!='.'):
            full_name = directory_name+fileNb
            img=mpimg.imread(full_name)
            imgr = img_to_array(img)
            imgs.append(imgr)
            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
            sys.stdout.flush()    
    return np.asarray(imgs)

'''def dataGenerator(batch_size, data, labels, aug_dict, image_save_prefix ="image", mask_save_prefix="mask", seed=1):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)

    image_generator = image_datagen.flow(
        data,
        batch_size = batch_size,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow(
        labels,
        batch_size = batch_size,
        save_prefix  = mask_save_prefix,
        seed = seed)
    
    for (img,mask) in zip(image_generator, mask_generator):
        if(np.max(img) > 1):
          img = img / 255
          mask = mask /255
          mask[mask > 0.5] = 1
          mask[mask <= 0.5] = 0
        yield (img,mask)'''

# target_size = (256,256)
def dataGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, target_size,
                  image_color_mode = "rgb", mask_color_mode = "grayscale",
                  image_save_prefix  = "image",mask_save_prefix  = "mask",
                  flag_multi_class = False,num_class = 2,save_to_dir = None, seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(#a
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        if(np.max(img) > 1):
            img /= 255.
            mask/= 255.
            mask[mask > 0.5] = 1
            mask[mask <= 0.5] = 0
        yield (img,mask)

def uneti(input_size, pretrained_weights=None, verbose=True):
    inputs = Input(shape=input_size)

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    drop5 = Dropout(0.5)(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(drop5)
    
    conv6 = Conv2D(2048, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool5)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(2048, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)
    drop6 = Dropout(0.5)(conv6)
    
    up7 = Conv2D(1024, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop6))
    merge7 = concatenate([drop5,up7], axis = 3)
    conv7 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([drop4,up8], axis = 3)
    conv8 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv3,up9], axis = 3)
    conv9 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)

    up10 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv9))
    merge10 = concatenate([conv2,up10], axis = 3)
    conv10 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge10)
    conv10 = BatchNormalization()(conv10)
    conv10 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv10)
    conv10 = BatchNormalization()(conv10)

    up11 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv10))
    merge11 = concatenate([conv1,up11], axis = 3)
    conv11 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge11)
    conv11 = BatchNormalization()(conv11)
    conv11 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv11)
    conv11 = BatchNormalization()(conv11)
    conv11 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv11)
    conv12 = Conv2D(1, 1, activation = 'sigmoid')(conv11)

    model = Model(inputs = inputs, outputs = conv12)
    
    ##binary_crossentropy
    ##categorical_crossentropy
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    if(verbose == True):
        model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

**1. Load the data and split it into validation and training**

In [0]:
# unet400
#imgs = data_load(root_path+'data_rs/train/images/')
#gts = data_load(root_path+'data_rs/train/groundtruth/')

# unet256 nous
'''from PIL import Image
def data_load_resize(directory_name, size):
    filenames = os.listdir(directory_name)
    imgs = []
    for i, fileNb in enumerate(filenames):
        if(fileNb!='.DS_Store'):     
            im1 = Image.open(directory_name+fileNb)
            side = size
            im2 = im1.resize((side, side), Image.NEAREST)
            imgr = img_to_array(im2)
            imgs.append(imgr)
            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
            sys.stdout.flush()    
    return np.asarray(imgs)

imgs = data_load_resize(root_path+'data_rs/train/images/', 256)
gts = data_load_resize(root_path+'data_rs/train/groundtruth/', 256)

TEST_SIZE = 0.2
X_train, X_test, gt_train, gt_test = train_test_split(imgs, gts, test_size=TEST_SIZE, random_state=1)'''

#unet 256 git
#nothing car c'est fait dans le generator

'from PIL import Image\ndef data_load_resize(directory_name, size):\n    filenames = os.listdir(directory_name)\n    imgs = []\n    for i, fileNb in enumerate(filenames):\n        if(fileNb!=\'.DS_Store\'):     \n            im1 = Image.open(directory_name+fileNb)\n            side = size\n            im2 = im1.resize((side, side), Image.NEAREST)\n            imgr = img_to_array(im2)\n            imgs.append(imgr)\n            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))\n            sys.stdout.flush()    \n    return np.asarray(imgs)\n\nimgs = data_load_resize(root_path+\'data_rs/train/images/\', 256)\ngts = data_load_resize(root_path+\'data_rs/train/groundtruth/\', 256)\n\nTEST_SIZE = 0.2\nX_train, X_test, gt_train, gt_test = train_test_split(imgs, gts, test_size=TEST_SIZE, random_state=1)'

**2. Data augmentation and generators**

In [0]:
data_gen_args = dict(rotation_range=0.15,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

#checkpoint_path = "training_1/cp.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                 save_weights_only=True,
#                                                 verbose=1)

# Create a callback that saves the model's weights
filepath = "weights.{epoch:02d}-{acc:.2f}.hdf5"
cp_callback = ModelCheckpoint(filepath=filepath, verbose=1, save_weights_only=True, period=1)

train_generator = dataGenerator(2,root_path+'data_rs/train','images','groundtruth',data_gen_args,(512,512))
#train_generator = dataGenerator(2,'data/train','images','groundtruth',data_gen_args,(256,256))


i = 0
for (img,mask) in myGene: 
    i = i+1
    print(i, img.shape, mask.shape)

**3. Load the model**

In [0]:
model = uneti(input_size = (512,512,3), pretrained_weights=None, verbose=True)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 64) 256         conv2d_1[0][0]                   
_________________________________________________________________

**4. Train the model**

In [12]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=1000,
                    epochs=10, 
                    verbose=1,
                    callbacks=[cp_callback])
                    #validation_data = validation_generator,
                    #validation_steps = 700
                    



Epoch 1/10
Found 100 images belonging to 1 classes.
Found 100 images belonging to 1 classes.
1000/1000 [==============================] - 2112s 2s/step - loss: 0.2957 - acc: 0.8913

Epoch 00001: saving model to weights.01-0.89.hdf5
Epoch 2/10
1000/1000 [==============================] - 2083s 2s/step - loss: 0.1278 - acc: 0.9510

Epoch 00002: saving model to weights.02-0.95.hdf5
Epoch 3/10
1000/1000 [==============================] - 2080s 2s/step - loss: 0.0876 - acc: 0.9656

Epoch 00003: saving model to weights.03-0.97.hdf5
Epoch 4/10
1000/1000 [==============================] - 2079s 2s/step - loss: 0.0673 - acc: 0.9726

Epoch 00004: saving model to weights.04-0.97.hdf5
Epoch 5/10
1000/1000 [==============================] - 2081s 2s/step - loss: 0.0631 - acc: 0.9741

Epoch 00005: saving model to weights.05-0.97.hdf5
Epoch 6/10
1000/1000 [==============================] - 2079s 2s/step - loss: 0.0424 - acc: 0.9820

Epoch 00006: saving model to weights.06-0.98.hdf5
Epoch 7/10
1000/

### Save model to disk

In [14]:
# serialize model to JSON
OUTPUT_FILENAME = "Model_UNET_512_acc98"

model_json = model.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

Saved model to disk


____
____
____
### Predictions (don't run it on Colab)

**Load model** [Optional]

In [24]:
from keras.models import model_from_json
json_file = open('model_saved/Model_UNET_512_acc98.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

**Resize images in a folder** [Optional]

In [35]:
from PIL import Image
import os
path1 = "data/tmp/"
path2 = "data/test/"

filenames = os.listdir(path1)
for i, fileNb in enumerate(filenames):
    if(fileNb=='.DS_Store'):    
        continue
    im1 = Image.open(path1+fileNb)
    side = 512
    # use one of these filter options to resize the image
    im2 = im1.resize((side, side), Image.NEAREST)
    im2.save(path2+fileNb)

In [25]:
data_test = data_load('data/test/')

test_datagenerator  = ImageDataGenerator()
testGene  = test_datagenerator.flow(data_test, batch_size=1)

model = uneti(input_size = (512,512,3), pretrained_weights=None, verbose=False)

model.load_weights("model_saved/Model_UNET_512_acc98.h5")

results = model.predict(data_test,verbose=1)

50/50 [==============================] - 2996s 60s/step


In [36]:
from skimage import img_as_ubyte

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def saveResult(from_path, save_path, predictions, concat=True):
    list_im_name = sorted_aphanumeric(os.listdir(from_path))
    if(list_im_name[0]=='.DS_Store'):
        list_im_name=list_im_name[1:]
    for name, pred in zip(list_im_name, predictions):
        img = mpimg.imread(from_path+name)
        if(concat):
            if(len(pred.shape)==3 and pred.shape[2]==1):
                pred = pred.reshape(pred.shape[0],pred.shape[1])
            cimg = concatenate_images(img, pred)
            io.imsave(save_path+"/prediction"+name[4:], cimg)
        else:
            io.imsave(save_path+"/prediction"+name[4:], pred)

In [38]:
saveResult("data/test/", "data/pred_unet_512_98a", results, concat=False)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0